In [ ]:
pip install transformers

     |████████████████████████████████| 4.0 MB 5.2 MB/s 
     |████████████████████████████████| 6.6 MB 67.3 MB/s 
     |████████████████████████████████| 77 kB 5.9 MB/s 
     |████████████████████████████████| 596 kB 75.5 MB/s 
     |████████████████████████████████| 880 kB 72.4 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=f1dcfd806f69bfc2a312c70ef52d4669c798cc0b0f257af0c5a47ca75c006cfe
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import pandas as pd

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cpu


**Read in the data** 

In [ ]:
dataset = pd.read_csv("/content/drive/MyDrive/Equintel_Uma/Data3/dax_news_since_2019-01_to_2020-06.csv")

In [ ]:
dataset.head()

,Unnamed: 0,company,date,url,esg_topics,title,content
0,2698,1&1Drillisch,2019-01-09,https://www.marketscreener.com/news/Trade-deal...,"['BioengineeredFoods', 'Fraud']",Trade deal optimism boosts European shares as ...,The pan-European STOXX 600 <.STOXX > hit a thr...
1,3025,1&1Drillisch,2019-03-27,https://www.marketscreener.com/quote/stock/1-1...,"['BioengineeredFoods', 'Environment', 'Transpa...",1 & 1 Drillisch: Continues Course of Growth in...,DGAP-Ad-hoc: 1 & 1 Drillisch Aktiengesellschaf...
2,13641,1&1Drillisch,2019-10-17,https://www.marketscreener.com/quote/stock/DRO...,"['DataSecurity', 'GDPR', 'Compliance']",Dropsuite Limited Announces New Partnership wi...,Dropsuite Limited announced a new partnership ...
3,14128,1&1Drillisch,2019-12-09,https://www.marketscreener.com/quote/stock/UNI...,"['BioengineeredFoods', 'Environment']",United Internet AG: Public share buyback offer...,DGAP-Ad-hoc: United Internet AG / Key word ( s...
4,21363,1&1Drillisch,2019-12-16,https://www.securityweek.com/web-hosting-firm-...,"['DataSecurity', 'GDPR', 'Privacy']",Web Hosting Firm Slapped With $ 10 Million GDP...,$ 10 Million GDPR Fine Imposed on German Telco...


In [ ]:
dataset.shape

(21618, 7)

In [ ]:
dataset.reset_index
dataset.drop('Unnamed: 0',axis =1, inplace=True)
dataset.tail()


,company,date,url,esg_topics,title,content
21613,thyssenkruppAG,2020-06-16,https://aithority.com/technology/analytics/ava...,['BioengineeredFoods'],Avanade Named Strategic Go-to Microsoft Azure ...,"Avanade, the leading digital innovator on the ..."
21614,thyssenkruppAG,2020-06-24,https://www.finance.yahoo.com/news/kkr-cinven-...,"['BioengineeredFoods', 'WasteManagement', 'Rec...","KKR, Cinven Among Firms Weighing Unilever Tea ...",( Bloomberg) -- Some of the world’ s top buyou...
21615,thyssenkruppAG,2020-06-24,https://www.marketscreener.com/news/Lineage-Lo...,"['EnergyEfficiency', 'BioengineeredFoods']",Lineage Logistics: Recognized by the U.S. Depa...,Lineage recognized for optimizing the blast fr...
21616,thyssenkruppAG,2020-06-24,https://www.marketscreener.com/THYSSENKRUPP-AG...,"['Environment', 'Transparency']",thyssenkrupp: Elevator provides free cloud-bas...,"Jun 24, 2020 10:30 AM thyssenkrupp Elevator of..."
21617,thyssenkruppAG,2020-06-25,https://uk.finance.yahoo.com/news/global-polyl...,"['BioengineeredFoods', 'Petroleum']",The global polylactic acid market is projected...,KEY FINDINGS The global polylactic acid market...


In [ ]:
import re
ILLEGAL_CHARACTERS_RE = re.compile(r'[\000-\010]|[\013-\014]|[\016-\037]')
dataset = dataset.applymap(lambda x: ILLEGAL_CHARACTERS_RE.sub(r'', x) if isinstance(x, str) else x)

In [ ]:
dataset.to_excel('/content/drive/MyDrive/Equintel_Uma/Data3/dax_news_data_part3_reindexed.xlsx', encoding = 'UTF-8')

**Pre-processing of Raw Data** 

**Language Detection**

In [ ]:
!pip install spacy-langdetect

     |████████████████████████████████| 998 kB 4.5 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.7-py3-none-any.whl size=993431 sha256=61faaf58e82ef0381df993bdfa836b48dfabf69b1586ecfef9b7b741fe46ae0c
  Stored in directory: /root/.cache/pip/wheels/89/79/3b/9885ae7f4308f73c514f96d8574d40d7d8173a27731b674013
Successfully built langdetect


In [ ]:
def clean_otherlanguages(dataset):
  import langdetect
  document = dataset['title']
  languages_langdetect = []

  # find the language of each headline
  for line in document:
    try:
        result = langdetect.detect_langs(line)
        result = str(result[0])[:2]
    except:
        result = 'unknown'
    finally:
        languages_langdetect.append(result)

  dataset['languages_langdetect'] = languages_langdetect
  dataset['languages_langdetect'].value_counts()
  dataset.to_excel('/content/drive/MyDrive/Equintel_Uma/Results_09-05-2022/language_detection.xlsx')
  dataset = dataset[dataset['languages_langdetect'] == 'en'] #remove rows which are not English 
  dataset.to_excel('/content/drive/MyDrive/Equintel_Uma/Results_09-05-2022/otherlanguages_removed_dataset.xlsx')

In [ ]:
clean_otherlanguages(dataset)

**Clean URLs from Content**

In [ ]:
dataset = pd.read_excel("/content/drive/MyDrive/Equintel_Uma/Results_09-05-2022/otherlanguages_removed_dataset.xlsx")

In [ ]:
def remove_URLs(sentence,to_be_excluded):
    tokens = sentence.split(" ")
    tokens_filtered= [word for word in tokens if not word in to_be_excluded]
    return (" ").join(tokens_filtered)

In [ ]:
def clean_URLs(dataset):
  from spacy.lang.tokenizer_exceptions import URL_PATTERN

  document = dataset['content']
  filtered_content = []

  nlp = spacy.load("en_core_web_sm")

  custom_infixes = [URL_PATTERN[1:-1]] 
  nlp.tokenizer.infix_finditer = spacy.util.compile_infix_regex(custom_infixes).finditer

  for text in document: 
    to_be_excluded = []
    for t in nlp.tokenizer.explain(nlp(text).text): 
      if (t[0] == 'INFIX'):
        to_be_excluded.append('//'+t[1])
    filtered_text = remove_URLs(text,to_be_excluded)
    filtered_content.append(filtered_text)
  dataset['filtered_content'] = filtered_content
  dataset.drop('content', axis=1, inplace=True)
  dataset.to_excel('/content/drive/MyDrive/Equintel_Uma/Results_09-05-2022/URLs_removed_dataset.xlsx')

In [ ]:
import spacy
clean_URLs(dataset)

**Keep only unique URLs ( to remove repetitive content)**

In [ ]:
dataset = pd.read_excel("/content/drive/MyDrive/Equintel_Uma/Results_09-05-2022/URLs_removed_dataset.xlsx")
dataset.drop_duplicates(subset=['url'], inplace=True)
dataset.drop('Unnamed: 0',axis=1,inplace=True)
dataset.drop('Unnamed: 0.1',axis=1,inplace=True)
dataset.reset_index()
dataset.to_excel('/content/drive/MyDrive/Equintel_Uma/Results_09-05-2022/DuplicateURLs_removed.xlsx')

 **Find the sentiment of the news headline** 

In [ ]:
dataset = pd.read_excel("/content/drive/MyDrive/Equintel_Uma/Results_09-05-2022/DuplicateURLs_removed.xlsx")

In [ ]:
def find_sentiment_newsheadlines():
  from transformers import AutoModelForSequenceClassification,RobertaTokenizer
  model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/Equintel_Uma/Trained_Models/senti-model-05_04_22")
  tokenizer=RobertaTokenizer.from_pretrained("/content/drive/MyDrive/Equintel_Uma/Trained_Models/senti-model-05_04_22")
  from transformers import pipeline
  classifier = pipeline("sentiment-analysis",model=model,tokenizer=tokenizer)
  roberta = []
  for str in dataset['title']: 
    roberta.append(classifier(str))

  esg_sentiment = []
  esg_score = []

  for row in roberta:
    if row[0]['label'] == 'POSITIVE':
      esg_sentiment.append(1) 
    else:
      esg_sentiment.append(0)
    esg_score.append(row[0]['score'])

  result_df = pd.DataFrame({'title': dataset['title'],'date': dataset['date'],'esg_sentiment_headline':esg_sentiment})
  result_df.to_excel('/content/drive/MyDrive/Equintel_Uma/Results_09-05-2022/result1.xlsx')  


In [ ]:
find_sentiment_newsheadlines()

**Find the sentiment of the news content**

In [ ]:
dataset = pd.read_excel("/content/drive/MyDrive/Equintel_Uma/Results_09-05-2022/DuplicateURLs_removed_sentiment_content_1.xlsx")

In [ ]:
import spacy
def find_sentiment_newscontent():
  from transformers import AutoModelForSequenceClassification,RobertaTokenizer
  model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/Equintel_Uma/Trained_Models/senti-model-05_04_22")
  tokenizer=RobertaTokenizer.from_pretrained("/content/drive/MyDrive/Equintel_Uma/Trained_Models/senti-model-05_04_22")
  from transformers import pipeline
  classifier = pipeline("sentiment-analysis",model=model,tokenizer=tokenizer,max_length=512, truncation=True)
  
  nlp = spacy.load('en')
  sentiment_content = []

  for text in dataset['filtered_content']:
    sentiment = []
    tokens = nlp(text)

    count_pos = 0 
    count_neg = 0

    for sent in tokens.sents:
      sentence = sent.string.strip()
      sentiment.append(classifier(sentence))

    for item in sentiment: 
      if item[0]['label'] == 'POSITIVE': 
        count_pos += 1
      else: 
        count_neg += 1

      if (count_pos > count_neg): 
        sentiment_sent = 1
      else: 
        sentiment_sent = 0
  
    sentiment_content.append(count_pos/(count_pos+count_neg))
    sentiment_content_rounded = [ round(elem, 2) for elem in sentiment_content ]
    data_esg_sentiment_content = {'ESG_sentiment_content': sentiment_content_rounded}
    esg_df= pd.DataFrame(data_esg_sentiment_content)
    esg_df.to_excel('/content/drive/MyDrive/Equintel_Uma/Results_09-05-2022/result3.xlsx')

In [ ]:
find_sentiment_newscontent()

**Find the company name for each headline** 

In [1]:
from google.colab import files

In [2]:
!cp /content/drive/MyDrive/Colab\ Notebooks/MyModules/NER_Company_names.py /content

In [3]:
import NER_Company_names

In [ ]:
files.upload()

{}

In [4]:
import pandas as pd
dataset = pd.read_excel("/content/drive/MyDrive/Equintel_Uma/Results_09-05-2022/DuplicateURLs_removed_company_headlines_1.xlsx")
dataset.shape
to_be_analysed_title = dataset['title']
to_be_analysed_content = dataset['filtered_content']

In [5]:
def find_companynames_headlines():
  document = to_be_analysed_title
  #Read in the gazeeteer file having the list of companies to be searched for 
  df = pd.read_excel('/content/drive/MyDrive/Equintel_Uma/Company Name Recognition/dax_companies.xlsx')
  matching_companies = []
  for row in document:
    company_names=[]
    concept_form = NER_Company_names.search_concept_forms(row,df)
    surface_form = NER_Company_names.search_surface_forms(row,df)
    amb_form = NER_Company_names.search_ambiguous_forms(row,df)
    if concept_form != '':
      company_names.append(concept_form)
    if surface_form != '':
      company_names.append(surface_form)
    if amb_form !='':
      company_names.append(amb_form)     
    matching_companies.append(company_names) 
    matching_companies_df = pd.DataFrame({'Company Names from Headlines': matching_companies})
    matching_companies_df.to_excel('/content/drive/MyDrive/Equintel_Uma/Results_09-05-2022/result_company_names_headlines_2.xlsx') 

In [6]:
find_companynames_headlines()

**Find the company name for each content**

In [1]:
from google.colab import files

In [2]:
!cp /content/drive/MyDrive/Colab\ Notebooks/MyModules/NER_Company_names.py /content

In [3]:
import NER_Company_names

In [5]:
import pandas as pd
dataset = pd.read_excel("/content/drive/MyDrive/Equintel_Uma/Results_09-05-2022/DuplicateURLs_removed_company_content_3.xlsx")
dataset.shape
to_be_analysed_title = dataset['title']
to_be_analysed_content = dataset['filtered_content']

In [6]:
def find_companynames_content():
  document = to_be_analysed_content
  df = pd.read_excel('/content/drive/MyDrive/Equintel_Uma/Company Name Recognition/dax_companies.xlsx')
  #find company names
  matching_companies = []
  for row in document:
    company_names=[]
    concept_form = NER_Company_names.search_concept_forms(row,df)
    surface_form = NER_Company_names.search_surface_forms(row,df)
    amb_form = NER_Company_names.search_ambiguous_forms(row,df)
    if concept_form != '':
      company_names.append(concept_form)
    if surface_form != '':
      company_names.append(surface_form)
    if amb_form !='':
      company_names.append(amb_form)     
    matching_companies.append(company_names) 
    matching_companies_df = pd.DataFrame({'Company Names from Content': matching_companies})
    matching_companies_df.to_excel('/content/drive/MyDrive/Equintel_Uma/Results_09-05-2022/result_company_names_content_4.xlsx') 

In [7]:
find_companynames_content()

**Get unique company names**

In [ ]:
def get_Unique_Company_names(): 
  dataset = pd.read_excel("/content/drive/MyDrive/Equintel_Uma/Data/dax_unique_company_names.xlsx")
  unique_company_names = dataset.concept.unique().tolist()
  unique_companies_df = pd.DataFrame({'Company Names': unique_company_names})
  unique_companies_df.to_excel("/content/drive/MyDrive/Equintel_Uma/Data/Unique_company_names_new.xlsx")

In [ ]:
get_Unique_Company_names()

**Match the resulting companies with normalized company names and frequency of company names** 

In [10]:
unique_companies_df = pd.read_excel("/content/drive/MyDrive/Equintel_Uma/Data/Unique_company_names_new.xlsx")

In [11]:
unique_companies_names = unique_companies_df['Company Names']

In [12]:
def get_company_name(first_company,unique_companies_names):
  first_company = first_company.strip()
  first_company = first_company.replace(' ','')
  first_company = first_company.replace('\n','')
  first_company = first_company.replace('[','')
  first_company = first_company.replace(']','')
  first_company = first_company.replace("'",'')
  first_company = first_company.replace("_",'')
  if first_company == 'Telefonica': 
    first_company = 'Telefónica'
  if first_company == 'alstria': 
    return 'alstriaofficeREIT-AG'
  if first_company == 'mtu': 
    return 'MTUAeroEngines'
  if first_company == 'thyssenkrupp': 
    return 'thyssenkruppAG'  
  if first_company.islower():
    first_company = first_company.capitalize()
  first_company = first_company.strip()
  #make the translation for some exceptional names  
  if first_company == '': 
    return None
  if first_company == 'BMW':
    return 'bmw'
  if first_company == 'Software':
    return 'SoftwareAG'
  if first_company == 'AtossSoftware':
    return 'AtossSoftwareAG'
  if first_company == 'Indus':
    return 'IndusHoldingAG'
  if first_company == 'ServiceNow':
    return 'ServiceNow'
  if first_company == 'Kloeckner':
    return 'Klöckner&CoSE'
  
  for company in unique_companies_names:
    if (company.find(first_company) != -1):
      return company

**Get the frequency and normalized names for Headlines**

In [14]:
def convert_normalized_company_names_headlines(): 
  dataset = pd.read_excel("/content/drive/MyDrive/Equintel_Uma/Results_09-05-2022/Result_Sheet.xlsx")
  dataset['Company Names from Headlines'] = dataset['Company Names from Headlines'].apply(lambda x:x[1:-1].split(','))
  company_names = dataset['Company Names from Headlines']
  data = pd.DataFrame(company_names)
  match = []
  frequency = []
  companies = data['Company Names from Headlines']
  for comp in companies:
    inner_match = []
    freq_dict = {}
    while(comp != []):
      first_company,*comp = comp
      c_name = get_company_name(first_company,unique_companies_names)
      if (c_name not in inner_match) and (c_name != None):
        inner_match.append(c_name)
        freq_dict[c_name] = 1
      elif c_name!= None:
        freq_dict[c_name] += 1      
    match.append(inner_match)
    frequency.append(freq_dict)

  df = pd.DataFrame({'Normalized Company names from Headlines': match})
  df1 = pd.DataFrame({'Frequency of Company Names from Headlines':frequency})
  dataset=pd.concat([dataset,df,df1],axis=1)
  dataset.to_excel("/content/drive/MyDrive/Equintel_Uma/Results_09-05-2022/Result_Sheet1.xlsx")
  

In [15]:
convert_normalized_company_names_headlines()

**Get the frequency and normalized names for Content**

In [16]:
def convert_normalized_company_names_content(): 
  dataset = pd.read_excel("/content/drive/MyDrive/Equintel_Uma/Results_09-05-2022/Result_Sheet1.xlsx")
  dataset['Company Names from Content'] = dataset['Company Names from Content'].apply(lambda x:x[1:-1].split(','))
  company_names = dataset['Company Names from Content']
  data = pd.DataFrame(company_names)
  match = []
  frequency = []
  companies = data['Company Names from Content']
  for comp in companies:
    inner_match = []
    freq_dict = {}
    while(comp != []):
      first_company,*comp = comp
      c_name = get_company_name(first_company,unique_companies_names)
      if (c_name not in inner_match) and (c_name != None):
        inner_match.append(c_name)
        freq_dict[c_name] = 1
      elif c_name!= None:
        freq_dict[c_name] += 1      
    match.append(inner_match)
    frequency.append(freq_dict)

  df = pd.DataFrame({'Normalized Company names from Content': match})
  df1 = pd.DataFrame({'Frequency of Company Names from Content':frequency})
  dataset=pd.concat([dataset,df,df1],axis=1)
  dataset.drop('Unnamed: 0',axis=1,inplace=True)
  dataset.drop('Unnamed: 0.1',axis=1,inplace=True)
  dataset.to_excel("/content/drive/MyDrive/Equintel_Uma/Results_09-05-2022/Result_Sheet2.xlsx")

In [17]:
convert_normalized_company_names_content()

**Sentiment Aggregation**

In [ ]:
def sentiment_aggregation(company,df,from_date=None,to_date=None):
  sentiment = 0
  if (from_date>to_date):
    print("From-Date can't be smaller than To-date")
    return sentiment
  for index,row in df.iterrows():
    if (df['Frequency of Company Names from Content'][index] != '{}') and (pd.isna(df['Frequency of Company Names from Content'][index])==False) and (df['date'][index]>=from_date) and (df['date'][index]<=to_date):
      comp = df['Frequency of Company Names from Content'][index]
      #print(comp)
      comp = comp[1:-1]     
      comp_list = comp.split(',')
      while(comp_list != []):
        first_company,*comp_list = comp_list
        first_company = first_company.replace('[','')
        first_company = first_company.replace(']','')
        first_company = first_company.replace("'",'')
        first_company = first_company.replace(" ",'')
        if first_company != '':
          temp = first_company.split(':')
          company_name = temp[0]
          frequency = int(temp[1])              
        if company_name==company:
          sentiment = sentiment + 2*df['esg_sentiment_headline'][index] + 2*frequency
          sentiment = sentiment + df['esg_sentiment_content'][index] + frequency
    if (df['Frequency of Company Names from Headlines'][index] != '{}') and (pd.isna(df['Frequency of Company Names from Headlines'][index])==False) and (df['date'][index]>=from_date) and (df['date'][index]<=to_date):
      comp = df['Frequency of Company Names from Headlines'][index]
      #print(comp)
      comp = comp[1:-1]     
      comp_list = comp.split(',')
      while(comp_list != []):
        first_company,*comp_list = comp_list
        first_company = first_company.replace('[','')
        first_company = first_company.replace(']','')
        first_company = first_company.replace("'",'')
        first_company = first_company.replace(" ",'')
        if first_company != '':
          temp = first_company.split(':')
          company_name = temp[0]
          frequency = int(temp[1])              
        if company_name==company:
          sentiment = sentiment + 2*df['esg_sentiment_headline'][index] + 2*frequency
          sentiment = sentiment + df['esg_sentiment_content'][index] + frequency
  return sentiment
  

In [ ]:
from numpy import datetime64
from datetime import date

dataset = pd.read_excel("/content/drive/MyDrive/Equintel_Uma/Results_19-04-2022/Result_Sheet2.xlsx")
dataset['date'] = dataset['date'].astype(datetime64)
#dataset['Frequency of Company Names']= dataset['Frequency of Company Names'].astype(object)
from_date = date(2020, 12, 1)
to_date = date(2021, 4, 1) 
sentiment = sentiment_aggregation('AllianzSE',dataset,from_date,to_date)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


In [ ]:
print(sentiment)

670.5900000000004
